In [1]:
import keras
import tensorflow as tf
import numpy as np

Using TensorFlow backend.


In [2]:
# Note: data isn't in the repository

# loading data
data = np.load("data/x_images.npy")
labels = np.load("data/y_images.npy")

# shuffling data
indices = np.random.choice(np.arange(len(data)), size=len(data), replace=False)
data = data[indices]
labels = labels[indices]

# encode labels to one-hot
num_classes = len(set((labels)))
labels = keras.utils.to_categorical(labels, num_classes)

print(data.shape)
print(labels.shape)

(234, 288, 352, 3)
(234, 8)


In [3]:
# load a pretrained model (without the top dense layers)

base_model = keras.applications.DenseNet121(include_top=False, weights='imagenet', input_tensor=None, input_shape=[288,352,3], pooling=None, classes=num_classes)

In [4]:
# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False


In [5]:
# https://keras.io/applications/
# add layers to the pretrained network
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.3)(x)
predictions = keras.layers.Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = keras.models.Model(inputs=base_model.input, outputs=predictions)

In [6]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=1e-5),
              metrics=['acc'])
# Train the model
history = model.fit(x=data,y=labels,batch_size=20,epochs=500,validation_split=0.3)


Train on 163 samples, validate on 71 samples
Epoch 1/500
163/163 [==============================] - 6s 36ms/step - loss: 2.4706 - acc: 0.1288 - val_loss: 5.2567 - val_acc: 0.2676
Epoch 2/500
163/163 [==============================] - 1s 8ms/step - loss: 2.3196 - acc: 0.1288 - val_loss: 5.1016 - val_acc: 0.2254
Epoch 3/500
163/163 [==============================] - 1s 8ms/step - loss: 2.1664 - acc: 0.1472 - val_loss: 4.8240 - val_acc: 0.2254
Epoch 4/500
163/163 [==============================] - 1s 8ms/step - loss: 2.1837 - acc: 0.1104 - val_loss: 4.6137 - val_acc: 0.2254
Epoch 5/500
163/163 [==============================] - 1s 8ms/step - loss: 2.1406 - acc: 0.0982 - val_loss: 4.4232 - val_acc: 0.2254
Epoch 6/500
163/163 [==============================] - 1s 8ms/step - loss: 2.0924 - acc: 0.1840 - val_loss: 4.2877 - val_acc: 0.2254
Epoch 7/500
163/163 [==============================] - 1s 8ms/step - loss: 1.9680 - acc: 0.2577 - val_loss: 4.2671 - val_acc: 0.2254
Epoch 8/500
163/163 [==

Epoch 62/500
163/163 [==============================] - 1s 8ms/step - loss: 1.3589 - acc: 0.5460 - val_loss: 4.5830 - val_acc: 0.3944
Epoch 63/500
163/163 [==============================] - 1s 8ms/step - loss: 1.3689 - acc: 0.4724 - val_loss: 4.6930 - val_acc: 0.3944
Epoch 64/500
163/163 [==============================] - 1s 8ms/step - loss: 1.2844 - acc: 0.5890 - val_loss: 4.8590 - val_acc: 0.3944
Epoch 65/500
163/163 [==============================] - 1s 8ms/step - loss: 1.3380 - acc: 0.5031 - val_loss: 4.9153 - val_acc: 0.3944
Epoch 66/500
163/163 [==============================] - 1s 8ms/step - loss: 1.2948 - acc: 0.5767 - val_loss: 4.8747 - val_acc: 0.3944
Epoch 67/500
163/163 [==============================] - 1s 8ms/step - loss: 1.2554 - acc: 0.5583 - val_loss: 4.8012 - val_acc: 0.3944
Epoch 68/500
100/163 [=================>............] - ETA: 0s - loss: 1.2797 - acc: 0.5100

KeyboardInterrupt: 

In [7]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

0 input_1
1 zero_padding2d_1
2 conv1/conv
3 conv1/bn
4 conv1/relu
5 zero_padding2d_2
6 pool1
7 conv2_block1_0_bn
8 conv2_block1_0_relu
9 conv2_block1_1_conv
10 conv2_block1_1_bn
11 conv2_block1_1_relu
12 conv2_block1_2_conv
13 conv2_block1_concat
14 conv2_block2_0_bn
15 conv2_block2_0_relu
16 conv2_block2_1_conv
17 conv2_block2_1_bn
18 conv2_block2_1_relu
19 conv2_block2_2_conv
20 conv2_block2_concat
21 conv2_block3_0_bn
22 conv2_block3_0_relu
23 conv2_block3_1_conv
24 conv2_block3_1_bn
25 conv2_block3_1_relu
26 conv2_block3_2_conv
27 conv2_block3_concat
28 conv2_block4_0_bn
29 conv2_block4_0_relu
30 conv2_block4_1_conv
31 conv2_block4_1_bn
32 conv2_block4_1_relu
33 conv2_block4_2_conv
34 conv2_block4_concat
35 conv2_block5_0_bn
36 conv2_block5_0_relu
37 conv2_block5_1_conv
38 conv2_block5_1_bn
39 conv2_block5_1_relu
40 conv2_block5_2_conv
41 conv2_block5_concat
42 conv2_block6_0_bn
43 conv2_block6_0_relu
44 conv2_block6_1_conv
45 conv2_block6_1_bn
46 conv2_block6_1_relu
47 conv2_block

In [7]:
for layer in model.layers[:245]:
   layer.trainable = False
for layer in model.layers[245:]:
   layer.trainable = True

In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(lr=1e-8),
              metrics=['acc'])
# Train the model
history = model.fit(x=data,y=labels,batch_size=20,epochs=30,validation_split=0.3)


Train on 163 samples, validate on 71 samples
Epoch 1/30
163/163 [==============================] - 9s 54ms/step - loss: 1.2674 - acc: 0.5521 - val_loss: 4.9690 - val_acc: 0.3944
Epoch 2/30
163/163 [==============================] - 2s 9ms/step - loss: 1.2762 - acc: 0.5153 - val_loss: 4.9684 - val_acc: 0.3944
Epoch 3/30
163/163 [==============================] - 2s 9ms/step - loss: 1.2791 - acc: 0.5399 - val_loss: 4.9658 - val_acc: 0.3944
Epoch 4/30
163/163 [==============================] - 2s 9ms/step - loss: 1.3047 - acc: 0.5460 - val_loss: 4.9637 - val_acc: 0.3944
Epoch 5/30
163/163 [==============================] - 2s 9ms/step - loss: 1.2712 - acc: 0.5337 - val_loss: 4.9620 - val_acc: 0.3944
Epoch 6/30
163/163 [==============================] - 2s 9ms/step - loss: 1.3034 - acc: 0.5399 - val_loss: 4.9542 - val_acc: 0.3944
Epoch 7/30
163/163 [==============================] - 2s 9ms/step - loss: 1.2458 - acc: 0.5337 - val_loss: 4.9355 - val_acc: 0.3944
Epoch 8/30
163/163 [==========

In [11]:
val_acc = str(int(history.history["val_acc"][-1]*100))
val_loss = str(int(history.history["val_loss"][-1]*10))
filename = '_val_acc_'+val_acc+"_val_loss_"+val_loss
# Save the weights
model.save_weights('data/model/weights_'+filename+'.h5')

# Save the model architecture
with open('data/model/model_ar_'+filename+'.json', 'w') as f:
    f.write(model.to_json())